In [259]:
from pulp import makeDict, LpProblem, LpMinimize, LpVariable, LpInteger, lpSum, LpStatus, value, LpMaximize, PULP_CBC_CMD
import numpy as np

In [260]:
Warehouses = ["A", "B"]
supply = {"A": 1000,
          "B": 4000}
Bars = ["1", "2", "3", "4", "5"]
demand = {"1":500,
          "2":900,
          "3":1800,
          "4":200,
          "5":700}

In [261]:
costs = [   #Bars
         #1 2 3 4 5
         [2,4,5,2,1],#A   Warehouses
         [3,1,3,2,3] #B
         ]


In [262]:
# The cost data is made into a dictionary
costs = makeDict([Warehouses,Bars],costs,0)
costs

defaultdict(<function pulp.utilities.__makeDict.<locals>.<lambda>()>,
            {'A': defaultdict(<function pulp.utilities.__makeDict.<locals>.<lambda>()>,
                         {'1': 2, '2': 4, '3': 5, '4': 2, '5': 1}),
             'B': defaultdict(<function pulp.utilities.__makeDict.<locals>.<lambda>()>,
                         {'1': 3, '2': 1, '3': 3, '4': 2, '5': 3})})

In [263]:
# Creates the 'prob' variable to contain the problem data
prob = LpProblem("Beer_Distribution_Problem",LpMinimize)

In [264]:
# Creates a list of tuples containing all the possible routes for transport
Routes = [(w,b) for w in Warehouses for b in Bars]
Routes

[('A', '1'),
 ('A', '2'),
 ('A', '3'),
 ('A', '4'),
 ('A', '5'),
 ('B', '1'),
 ('B', '2'),
 ('B', '3'),
 ('B', '4'),
 ('B', '5')]

In [265]:
# A dictionary called 'Vars' is created to contain the referenced variables(the routes)
vars = LpVariable.dicts("Route",(Warehouses,Bars), 0, None, LpInteger)
vars

{'A': {'1': Route_A_1,
  '2': Route_A_2,
  '3': Route_A_3,
  '4': Route_A_4,
  '5': Route_A_5},
 'B': {'1': Route_B_1,
  '2': Route_B_2,
  '3': Route_B_3,
  '4': Route_B_4,
  '5': Route_B_5}}

In [266]:
# The objective function is added to 'prob' first
prob += lpSum([vars[w][b]*costs[w][b] for (w,b) in Routes]), "Sum_of_Transporting_Costs"

In [267]:
# The demand minimum constraints are added to prob for each demand node (bar)
for b in Bars:
    prob += lpSum([vars[w][b] for w in Warehouses])>=demand[b], "Sum_of_Products_into_Bar%s"%b

In [268]:
prob

Beer_Distribution_Problem:
MINIMIZE
2*Route_A_1 + 4*Route_A_2 + 5*Route_A_3 + 2*Route_A_4 + 1*Route_A_5 + 3*Route_B_1 + 1*Route_B_2 + 3*Route_B_3 + 2*Route_B_4 + 3*Route_B_5 + 0
SUBJECT TO
Sum_of_Products_into_Bar1: Route_A_1 + Route_B_1 >= 500

Sum_of_Products_into_Bar2: Route_A_2 + Route_B_2 >= 900

Sum_of_Products_into_Bar3: Route_A_3 + Route_B_3 >= 1800

Sum_of_Products_into_Bar4: Route_A_4 + Route_B_4 >= 200

Sum_of_Products_into_Bar5: Route_A_5 + Route_B_5 >= 700

VARIABLES
0 <= Route_A_1 Integer
0 <= Route_A_2 Integer
0 <= Route_A_3 Integer
0 <= Route_A_4 Integer
0 <= Route_A_5 Integer
0 <= Route_B_1 Integer
0 <= Route_B_2 Integer
0 <= Route_B_3 Integer
0 <= Route_B_4 Integer
0 <= Route_B_5 Integer

In [269]:
# The problem is solved using PuLP's choice of Solver
prob.solve()

1

In [270]:
# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])

Status: Optimal


In [271]:
# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    print(v.name, "=", v.varValue)

Route_A_1 = 500.0
Route_A_2 = 0.0
Route_A_3 = 0.0
Route_A_4 = 200.0
Route_A_5 = 700.0
Route_B_1 = 0.0
Route_B_2 = 900.0
Route_B_3 = 1800.0
Route_B_4 = 0.0
Route_B_5 = 0.0


In [272]:
# The optimised objective function value is printed to the screen    
print("Total Cost of Transportation = ", value(prob.objective))

Total Cost of Transportation =  8400.0


## Boiden college example

In [273]:
lp = LpProblem("Bakery_Problem", LpMaximize)

In [274]:
#Define a dictionary of variables keyed by "indices"
var_keys = [1,2] 
x = LpVariable.dicts("Bakery_item", var_keys, lowBound=0, cat="Integer")
print(x)

{1: Bakery_item_1, 2: Bakery_item_2}


In [275]:
type(x)

dict

In [276]:
#Add the objective function
lp += 10* x[1] + 5 * x[2]

In [277]:
# Add the constraints
#lp += (5 * x[1] + x[2] <= 90, "oven_constraint")
#lp += ( lpSum( [5*x[1], x[2]] ) <= 90  )
lp += (x[1] + 10 * x[2] <= 300, "food_processor_constraint")
lp += (4 * x[1] + 6 * x[2] <= 125, "boiler_constraint")

In [278]:
#Rewrite the first constraint:
coeff = [5, 1] #may come from a file
coeff_dict = dict( zip(var_keys,coeff) )
lp += ( lpSum(coeff_dict[i] * x[i] for i in var_keys) <= 90, 'oven_const')

In [279]:
lp

Bakery_Problem:
MAXIMIZE
10*Bakery_item_1 + 5*Bakery_item_2 + 0
SUBJECT TO
food_processor_constraint: Bakery_item_1 + 10 Bakery_item_2 <= 300

boiler_constraint: 4 Bakery_item_1 + 6 Bakery_item_2 <= 125

oven_const: 5 Bakery_item_1 + Bakery_item_2 <= 90

VARIABLES
0 <= Bakery_item_1 Integer
0 <= Bakery_item_2 Integer

In [280]:
# Solve the LP
status = lp.solve(PULP_CBC_CMD(msg=0))
print("Status:", status)

Status: 1


In [281]:
#Print solution
for var in lp.variables():
    print(var, "=", value(var))
print("OPT =", value(lp.objective))

Bakery_item_1 = 16.0
Bakery_item_2 = 10.0
OPT = 210.0


In [282]:
for var in lp.variables():
    print(var, value(var))

Bakery_item_1 16.0
Bakery_item_2 10.0


In [283]:
value(lp.objective)

210.0

## Try it out

In [284]:
lin_pro = LpProblem("SC_Problem", LpMaximize)

In [285]:
from graph_ex import sc

In [286]:
#Define a dictionary of variables keyed by "indices"
opt_tuples = sc.get_optimising_tuples()
names = [str(val1)+"__"+str(val2) for val1, val2 in opt_tuples]
_x = [LpVariable(name=names[i], lowBound=0, cat='Integer') for i,_ in enumerate(names)] 


In [287]:
#Add the objective function

trans_cost = sc.get_transport_cost()
prices = sc.get_price_per_product()
supply_cost = sc.get_supply_costs()

lin_pro += lpSum(prices[i] * _x[i] - trans_cost[i] * _x[i] - supply_cost[i] * _x[i] for i,_ in enumerate(_x))

### Constraints

In [288]:
def add_demand_constraint(demand_val:np.int64, indexs:list, cprod_name:str)-> None:
    ''' Takes a demand value and list of indices and adds a constraint that the 
        demand value <= sum of index values'''
    return lpSum(_x[i] for i in indexs) <= demand_val, f'{cprod_name}_demand_constraint'

In [289]:
# Demand constraints
# lp += ( lpSum(coeff_dict[i] * x[i] for i in var_keys) <= 90, 'oven_const')
cprods = [cprod for cust in sc.customers.keys() for cprod in sc.graph.predecessors(cust)]
demand = sc.get_demand_vec()

for dem, cprod in zip(demand, cprods):
    indices = [names.index(fprod_cprod) for fprod_cprod in names if fprod_cprod.endswith(cprod)]
    lin_pro += add_demand_constraint(demand_val=dem, indexs=indices, cprod_name=cprod)
    

In [291]:
# lin_pro